In [ ]:
import bm_utils

paths = ["../preprocessing/tensors/2023_6less.npz",
         "../preprocessing/tensors/2024_6less.npz"]
X_train, X_test, Y_train, Y_test = bm_utils.load_dataset(paths, test_size=0.05, random_state=42)
print(len(X_train), len(X_test))

In [ ]:
import sys
sys.path.insert(1, '../preprocessing/')
import utils
import chess
import chess.svg
import cairosvg
from PIL import Image
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import io
from tqdm import tqdm

# Assume fens is your list of 100+ FEN strings
fens = []
for i in range(100):
    x = X_test[i]
    board = utils.tensor_to_board(x)
    board.turn = chess.BLACK
    fens.append(board.fen())

# PDF setup
pdf_file = "benchmark100.pdf"
c = canvas.Canvas(pdf_file, pagesize=letter)
page_width, page_height = letter

# Layout setup
cols = 2
rows = 3
margin = 50
board_width = 200
board_height = 200
x_spacing = (page_width - 2 * margin - cols * board_width) / (cols - 1)
y_spacing = (page_height - 2 * margin - rows * board_height) / (rows - 1)
title_height = 15

for idx, fen in tqdm(enumerate(fens)):
    col = idx % cols
    row = (idx // cols) % rows
    page = idx // (cols * rows)

    if idx % (cols * rows) == 0 and idx != 0:
        c.showPage()

    # Set position
    x_pos = margin + col * (board_width + x_spacing)
    y_pos = page_height - margin - (row + 1) * board_height - row * y_spacing

    # Parse board, flipped for black
    board = chess.Board(fen)
    svg_image = chess.svg.board(board=board, coordinates=True, flipped=True)

    # Convert to PNG
    png_bytes = io.BytesIO()
    cairosvg.svg2png(bytestring=svg_image.encode('utf-8'), write_to=png_bytes)
    png_bytes.seek(0)

    # Wrap for ReportLab
    pil_image = Image.open(png_bytes)
    image_reader = ImageReader(pil_image)

    # Draw title
    c.setFont("Helvetica", 12)
    c.drawCentredString(x_pos + board_width / 2, y_pos + board_height + title_height, f"Position #{idx + 1}")

    # Draw board image
    c.drawImage(image_reader, x_pos, y_pos, width=board_width, height=board_height)

# c.save()

In [ ]:
answers = []
for i in range(len(X_test[:100])):
    x = X_test[i]
    y = Y_test[i]
    
    board = utils.tensor_to_board(x)
    
    move = utils.tensor_to_move(y)
    san = board.san(move)
    
    answers.append(f"Position #{i + 1}: {san}")

# Add final page with all the answers
c.showPage()
c.setFont("Helvetica", 12)
line_height = 15
x_margin = 50
y_position = page_height - 50

for line in answers:
    c.drawString(x_margin, y_position, line)
    y_position -= line_height
    if y_position < 50:
        c.showPage()
        y_position = page_height - 50
        c.setFont("Helvetica", 12)

c.save()